# Tenor saxophone

* Sample duration = 5s
* model = SVM
* target = TENOR

Distinguish between samples with tenor saxophone from those without tenor saxophone. Other reed instruments excluded. Brass instruments not excluded.

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA 
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, \
                                    RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import BaggingClassifier
from scipy.stats import uniform

In [4]:
df = pd.read_pickle('./data/5s/labeled/features_r02/all_data.pkl')

In [5]:
NUM_LABEL_COLS = 13
NUM_COMPONENTS = 34
TEST_SIZE = 0.2
RANDOM_SEED = 0

In [6]:
df_filtered = df
df_filtered.shape

(9080, 10790)

In [8]:
# exclude records we want to exclude
df_filtered = df_filtered[df_filtered['alto'] == '0'] 
df_filtered = df_filtered[df_filtered['sop'] == '0']
df_filtered = df_filtered[df_filtered['bari'] == '0']
df_filtered = df_filtered[df_filtered['clrt'] == '0']
df_filtered = df_filtered[df_filtered['tora'] == '0']
df_filtered = df_filtered[df_filtered['othr'] == '0']

num_x_cols = df_filtered.shape[1] - NUM_LABEL_COLS  
# ^ last bit to adjust for zero indexing
data = df_filtered.iloc[:, 1:num_x_cols].to_numpy() 
# ^ These are the features
scaler = StandardScaler()
scaler.fit(data)
data = scaler.transform(data)

target = df_filtered[['tenr']].to_numpy().ravel()  # << This is the label

print(data.shape)
print(target.shape)

(7594, 10776)
(7594,)


In [9]:
pca = PCA(n_components=NUM_COMPONENTS)

In [10]:
pca.fit(data)

PCA(n_components=34)

In [11]:
d = pca.transform(data)

In [12]:
x_train, x_test, y_train, y_test = \
     train_test_split(d, target, test_size=TEST_SIZE, random_state=RANDOM_SEED)

## Test plain vanilla SVM

In [13]:
clf = SVC()
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(clf.score(x_test, y_test))

0.8426596445029625


In [14]:
c_matrix = confusion_matrix(y_test, y_pred)
idx = cols = [0, 1]
pd.DataFrame(c_matrix, index=idx, columns=cols)

,0,1
0,668,124
1,115,612


In [15]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.84      0.85       792
           1       0.83      0.84      0.84       727

    accuracy                           0.84      1519
   macro avg       0.84      0.84      0.84      1519
weighted avg       0.84      0.84      0.84      1519



## Determine good value for C using random search

In [16]:
model_params = {
    'C': uniform(0.1, 10.0)
}

# RandomizedSearchCV(estimator, param_distributions, *, n_iter=10, 
#                    scoring=None, n_jobs=None, iid='deprecated', 
#                    refit=True, cv=None, verbose=0, 
#                    pre_dispatch='2*n_jobs', random_state=None, 
#                    error_score=nan, return_train_score=False)
svm_model = SVC()
clf = RandomizedSearchCV(svm_model, model_params, n_iter=10, 
                         cv=5)
model = clf.fit(x_train, y_train)
params = model.best_estimator_.get_params()

In [17]:
params

{'C': 4.414985513905948,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [18]:
clf = SVC(C=params['C'])
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(clf.score(x_test, y_test))


0.8551678736010533


In [19]:
c_matrix = confusion_matrix(y_test, y_pred)
idx = cols = [0, 1]
pd.DataFrame(c_matrix, index=idx, columns=cols)

,0,1
0,673,119
1,101,626


In [20]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.85      0.86       792
           1       0.84      0.86      0.85       727

    accuracy                           0.86      1519
   macro avg       0.85      0.86      0.86      1519
weighted avg       0.86      0.86      0.86      1519



## Test ensemble approach with bagging classifier

#### Determine good parameters to use with bagging classifier. Start with grid search.

In [21]:
# Parameters.
p = {'max_features': [NUM_COMPONENTS - 2, NUM_COMPONENTS - 3, 
                      NUM_COMPONENTS - 4, NUM_COMPONENTS - 5,
                      NUM_COMPONENTS - 6, NUM_COMPONENTS - 7,
                      NUM_COMPONENTS - 8, NUM_COMPONENTS - 9,
                      NUM_COMPONENTS - 10, NUM_COMPONENTS - 11,
                      NUM_COMPONENTS - 12, NUM_COMPONENTS - 13,
                      NUM_COMPONENTS - 14, NUM_COMPONENTS - 15]}

base_estimator = SVC(C=params['C'])
estimator = BaggingClassifier(base_estimator=base_estimator, n_estimators=100)
# Question: Should we include C here, or ... ? There's no reason it should differ
# for different values of n_estimators, but what about max_features? Might different
# values for C be best for different max_features? Maybe. Dunno. Maybe we should
# search for optimal C *after* we've found the best value for max_features?

# GridSearchCV(estimator, param_grid, *, scoring=None, n_jobs=None, 
#              iid='deprecated', refit=True, cv=None, verbose=0, 
#              pre_dispatch='2*n_jobs', error_score=nan, return_train_score=False)
clf = GridSearchCV(estimator, p, n_jobs=-1)
search = clf.fit(x_train, y_train)
search.best_params_

# {'max_features': 24}

{'max_features': 24}

#### Run bagging classifier with good params

In [23]:
# BaggingClassifier(base_estimator=None, n_estimators=10, *, 
#                   max_samples=1.0, max_features=1.0, 
#                   bootstrap=True, bootstrap_features=False, 
#                   oob_score=False, warm_start=False, 
#                   n_jobs=None, random_state=None, verbose=0) 
clf = BaggingClassifier(base_estimator=SVC(C=params['C']),
                        n_estimators=500, 
                        max_features=search.best_params_['max_features'],
                        n_jobs=-1,
                        random_state=RANDOM_SEED)
model = clf.fit(x_train, y_train)

In [24]:
y_pred = clf.predict(x_test)
print(clf.score(x_test, y_test))

0.8538512179065174


In [25]:
c_matrix = confusion_matrix(y_test, y_pred)
idx = cols = [0, 1]
pd.DataFrame(c_matrix, index=idx, columns=cols)

,0,1
0,680,112
1,110,617


In [26]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.86      0.86       792
           1       0.85      0.85      0.85       727

    accuracy                           0.85      1519
   macro avg       0.85      0.85      0.85      1519
weighted avg       0.85      0.85      0.85      1519



### Try weakening our learners. Maybe bagging will work better then.

* Reduce training set size (increase test set size)
* Do not optimize C
* Explore wider range of parameters in grid search

In [26]:
x_train, x_test, y_train, y_test = \
     train_test_split(d, target, test_size=0.4, random_state=RANDOM_SEED)


In [27]:
# Parameters
p = {'n_estimators': [100, 200, 300, 400, 500, 600],
     'max_features': [NUM_COMPONENTS - 2, NUM_COMPONENTS - 3, 
                      NUM_COMPONENTS - 4, NUM_COMPONENTS - 5,
                      NUM_COMPONENTS - 6, NUM_COMPONENTS - 7]}

base_estimator = SVC()  # do not pass in C
estimator = BaggingClassifier(base_estimator=base_estimator)

# GridSearchCV(estimator, param_grid, *, scoring=None, n_jobs=None, 
#              iid='deprecated', refit=True, cv=None, verbose=0, 
#              pre_dispatch='2*n_jobs', error_score=nan, return_train_score=False)
clf = GridSearchCV(estimator, p, n_jobs=-1)
search = clf.fit(x_train, y_train)
search.best_params_

# {'max_features': 24, 'n_estimators': 100}

{'max_features': 28, 'n_estimators': 100}

In [28]:
# BaggingClassifier(base_estimator=None, n_estimators=10, *, 
#                   max_samples=1.0, max_features=1.0, 
#                   bootstrap=True, bootstrap_features=False, 
#                   oob_score=False, warm_start=False, 
#                   n_jobs=None, random_state=None, verbose=0) 
clf = BaggingClassifier(base_estimator=SVC(),
                        n_estimators=search.best_params_['n_estimators'], 
                        max_features=search.best_params_['max_features'],
                        random_state=RANDOM_SEED)
model = clf.fit(x_train, y_train)

In [29]:
y_pred = clf.predict(x_test)
print(clf.score(x_test, y_test))

0.8403284671532847


In [30]:
c_matrix = confusion_matrix(y_test, y_pred)
idx = cols = [0, 1]
pd.DataFrame(c_matrix, index=idx, columns=cols)

,0,1
0,996,158
1,192,846


In [31]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.86      0.85      1154
           1       0.84      0.82      0.83      1038

    accuracy                           0.84      2192
   macro avg       0.84      0.84      0.84      2192
weighted avg       0.84      0.84      0.84      2192

